<a href="https://colab.research.google.com/github/Phatdat01/recommendation/blob/dev/temp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install mlflow
!pip install gdown
!pip install pyspark
!gdown 10xqEKqtA04JSKU2rF0XZQP9X34ovtgoY

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 48 kB/s 
     |████████████████████████████████| 199 kB 65.6 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=8dd026008fd5e0e73efa5bfed2bedc3c80c945d0cec5c47338360e89f686def3
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark
Downloading...
From: https://drive.google.com/uc?id=10xqEKqtA04JSKU2rF0XZQP9X34ovtgoY
To: /content/yelp_dataset.tar
100% 4.34G/4.34G [00:20<00:00, 212MB/s]


In [ ]:
!pip install --upgrade --no-cache-dir gdown
!python -m gdown.cli "https://drive.google.com/uc?id=10xqEKqtA04JSKU2rF0XZQP9X34ovtgoY&confirm=t"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Attempting uninstall: gdown
    Found existing installation: gdown 4.4.0
    Uninstalling gdown-4.4.0:
      Successfully uninstalled gdown-4.4.0
Downloading...
From: https://drive.google.com/uc?id=10xqEKqtA04JSKU2rF0XZQP9X34ovtgoY&confirm=t
To: /content/yelp_dataset.tar
100% 4.34G/4.34G [00:44<00:00, 98.4MB/s]


In [ ]:
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.ml.recommendation import ALSModel
from pyspark.sql import Row
import numpy as np
from datetime import datetime
import tarfile
import matplotlib.pyplot as plt
import pandas as pd
from pyspark.sql import Window
from pyspark.sql.window import *

In [ ]:
%cd /content
# !mkdir yelp_dataset
my_tar = tarfile.open('/content/yelp_dataset.tar')
my_tar.extract('yelp_academic_dataset_review.json',".")
my_tar.close()
!rm /content/yelp_dataset.tar

/content


In [ ]:
spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '5000')\
        .getOrCreate()

In [ ]:
df = spark.read.json("/content/yelp_academic_dataset_review.json")
df.printSchema()
df.show(5)

In [ ]:
ratings=df.select("user_id","business_id","stars","date","cool","funny","useful")

In [ ]:
userRatings=ratings.groupBy("user_id").count()
## take id type int
window = Window.orderBy(col('user_id'))
userRatings = userRatings.withColumn('userid', row_number().over(window))

##Because ALS in spark require users or items is  in int type, we must process it

## if you use this, for the large data, it processes wrong
# userRatings=ratings.groupBy("user_id").count().withColumn("userid", monotonically_increasing_id().cast(IntegerType()))
userRatings.show(5)

In [ ]:
buiRatings=ratings.groupBy("business_id").count()
## take id type int
window = Window.orderBy(col('business_id'))
buiRatings = buiRatings.withColumn('businessid', row_number().over(window))

# buiRatings=ratings.groupBy("business_id").count().withColumn("businessid", monotonically_increasing_id().cast(IntegerType()))
newratings=ratings.join(userRatings, ['user_id'])
newratings=newratings.join(buiRatings, ['business_id'])
newratings=newratings.select("userid","businessid","stars","date")

In [ ]:
newratings=newratings.withColumn("stars",col("stars").cast(IntegerType()))
newratings=newratings.withColumn("date",to_timestamp("date"))

print(newratings)
newratings.show(5)

In [ ]:
num=int(newratings.count()*0.3)
ortherDate=newratings.orderBy(col("date").desc())
test=ortherDate.limit(int(num))
train=ortherDate.subtract(test)

In [ ]:
userTrain=train.groupBy('userid').count().select('userid')
## remove same id 
duplicate=test.join(userTrain,['userid'])
new=test.subtract(duplicate)

In [ ]:
def SplitTest(source,data):
    train,test=data.randomSplit([0.3,0.7])
    try:
        return source.union(train)
    except:
        return train

In [ ]:
## Keep data test with keep
newTrain = None
userNew=new.groupBy('userid').count().select('userid')
lstuserNew=userNew.rdd.flatMap(lambda x: x).collect()

for i in lstuserNew:
    userI=new.filter(col('userid')==i)
    newTrain=SplitTest(newTrain,userI)
    

In [ ]:
## remove test data to union to train and for test
numNewTrain=newTrain.count()
print(numNewTrain)
descDate=train.orderBy(col("date").desc())
forTest=descDate.limit(numNewTrain)

newtrain=train.subtract(forTest).union(newTrain)
newTest=test.subtract(newTrain).union(forTest)

In [ ]:
newTest.show(20)

In [ ]:
val,tes=newTest.randomSplit([2/3,1/3])
print(f"num newtest: {newTest}")
print(f"num val: {val}")
print(f"num tes: {tes}")

In [ ]:
als= ALS(rank=4, maxIter=4, regParam=0.35, userCol="userid", itemCol="businessid", ratingCol="stars", nonnegative=True,implicitPrefs = False,coldStartStrategy="drop")
model = als.fit(newtrain)
predictions=model.transform(newtrain)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="stars", predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print(f"training RMSE={rmse}")

predictions=model.transform(val)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="stars", predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print(f"validate RMSE={rmse}")

In [ ]:
predictions=model.transform(tes)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="stars", predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print(f"validate RMSE={rmse}")